In [16]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import os

try:
    connection = mysql.connector.connect(
        host='quote.c9ac6sewqau0.ap-southeast-2.rds.amazonaws.com',
        database='quote',
        user='admin',
        password='admin123'
    )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)
# Table name is Tradeasy_quotation

Connected to MySQL Server version  8.0.35


In [17]:
def create_table(connection):
    try:
        cursor = connection.cursor()
        create_table_query = '''CREATE TABLE IF NOT EXISTS Tradeasy_quotation (
            id INT AUTO_INCREMENT PRIMARY KEY,
            productName VARCHAR(255),
            productTag VARCHAR(255),
            supplier VARCHAR(255),
            category ENUM('Beef', 'Pork', 'Chicken', 'Poultry', 'Lamb', 'Fish', 'Seafood', 'Shrimp', 'Meatballs', 'Premade', 'Vegetables', 'Retail', 'Others'),
            packing VARCHAR(255),
            origin VARCHAR(255),
            brand VARCHAR(255),
            effectiveDate DATE,
            spec1 VARCHAR(255),
            spec2 VARCHAR(255),
            spec3 VARCHAR(255),
            spec4 VARCHAR(255),
            spec5 VARCHAR(255),
            spec6 VARCHAR(255),
            price DECIMAL(10, 2),
            weightUnit ENUM('KG', 'LB', 'PC', 'CTN'),
            warehouse ENUM('嘉里温控貨倉1', '嘉里温控貨倉2', '沙田冷倉1倉', '沙田冷倉2倉', '其士冷藏倉庫', '光輝1倉', '光輝2倉', '威強凍倉', '亞洲生活冷倉', '嘉威倉', '百匯倉', '萬集倉', '萬安倉', '送貨'),
            notes TEXT
        );'''
        cursor.execute(create_table_query)
        print("Table created successfully")
    except mysql.connector.Error as error:
        print(f"Failed to create table in MySQL: {error}")
    finally:
        if connection.is_connected():
            cursor.close()

def insert_product(connection, product_details):
    try:
        cursor = connection.cursor()
        insert_query = '''INSERT INTO Tradeasy_quotation (productName, productTag, supplier, category, packing, origin, brand, effectiveDate, spec1, spec2, spec3, spec4, spec5, spec6, price, weightUnit, warehouse, notes)
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
        cursor.execute(insert_query, product_details)
        connection.commit()
        print("Product inserted successfully")
    except mysql.connector.Error as error:
        print(f"Failed to insert product: {error}")
    finally:
        if connection.is_connected():
            cursor.close()

def delete_product(connection, product_id):
    try:
        cursor = connection.cursor()
        delete_query = "DELETE FROM Tradeasy_quotation WHERE id = %s;"
        cursor.execute(delete_query, (product_id,))
        connection.commit()
        print("Product deleted successfully")
    except mysql.connector.Error as error:
        print(f"Failed to delete product: {error}")
    finally:
        if connection.is_connected():
            cursor.close()

def update_product(connection, product_details):
    try:
        cursor = connection.cursor()
        update_query = '''UPDATE Tradeasy_quotation
                          SET productName = %s, price = %s, notes = %s
                          WHERE id = %s;'''
        cursor.execute(update_query, product_details)
        connection.commit()
        print("Product updated successfully")
    except mysql.connector.Error as error:
        print(f"Failed to update product: {error}")
    finally:
        if connection.is_connected():
            cursor.close()

def query_data(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        records = cursor.fetchall()  # Fetch all records from the last executed statement

        print("Total number of rows in table: ", cursor.rowcount)
        print("\nPrinting each row")

        for row in records:
            print("Product ID: ", row[0])
            print("Product Name: ", row[1])
            print("Category: ", row[3])
            print("Price: ", row[15])
            print("Warehouse: ", row[16])
            print("\n")

    except Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if connection.is_connected():
            cursor.close()


def query_data_dataframe(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        columns = cursor.column_names
        
        df = pd.DataFrame(result, columns=columns)
        return df

    except Error as e:
        print("Error reading data from MySQL table", e)
        return None
    finally:
        if connection.is_connected():
            cursor.close()

def dataframe_to_db(df, connection):
    for index, row in df.iterrows():
        product_details = tuple(row)  # Convert the DataFrame row to a tuple
        insert_product(connection, product_details)

In [5]:
#generate sample data 
create_table(connection)

Table created successfully


In [11]:
#insert sample data
sample_data = [
    ('Frozen Atlantic Salmon', 'Fish - Frozen', 'Oceanic Seafoods', 'Fish', 'Vacuum Packed', 'Norway', 'SeaKing', '2023-03-01', 'Wild-Caught', 'Skin-On', '2cm Thickness', '', '', '', 8.99, 'KG', '嘉里温控貨倉1', 'Ideal for grilling or baking.'),
    ('Organic Chicken Breast', 'Poultry - Fresh', 'Farmstead Poultry', 'Chicken', 'Plastic Wrapped', 'USA', 'FarmFresh', '2023-03-15', 'Organic', 'Boneless', 'Skinless', '', '', '', 5.49, 'LB', '沙田冷倉1倉', 'Perfect for healthy meals.'),
    ('Angus Beef Ribeye Steak', 'Beef - Premium', 'Quality Meats Ltd.', 'Beef', 'Vacuum Packed', 'Australia', 'BlackLabel', '2023-04-05', 'Grass-Fed', 'Angus', '1-inch Cut', 'Marble Score 4', '', '', 15.99, 'KG', '其士冷藏倉庫', 'Great for BBQ.'),
    ('Jumbo Tiger Shrimp', 'Seafood - Frozen', 'Blue Ocean', 'Seafood', 'Frozen', 'Vietnam', 'OceanBounty', '2023-02-20', 'Wild-Caught', 'Jumbo Size', 'Shell On', '', '', '', 12.50, 'LB', '亞洲生活冷倉', 'Delicious in pasta and stir-fries.'),
    ('Mixed Vegetables', 'Veggies - Frozen', 'Green Harvest', 'Vegetables', 'Bagged', 'Canada', 'VeggieDelight', '2023-05-01', 'Carrots', 'Peas', 'Corn', 'Green Beans', '', '', 2.99, 'KG', '百匯倉', 'Convenient for quick meals.'),
    ('Lamb Loin Chops', 'Lamb - Fresh', 'Shepherd’s Pride', 'Lamb', 'Plastic Wrapped', 'New Zealand', 'LambLux', '2023-06-10', 'Grass-Fed', '2cm Thickness', 'Bone In', '', '', '', 18.75, 'KG', '嘉威倉', 'Juicy and tender, perfect for grilling.'),
    ('Pork Sausages', 'Pork - Breakfast', 'Daily Deli', 'Pork', 'Vacuum Packed', 'Germany', 'MorningFarm', '2023-07-04', 'No Fillers', 'Gluten-Free', '', '', '', '', 4.99, 'LB', '光輝1倉', 'Great for breakfasts and BBQs.'),
    ('Italian Fusilli Pasta', 'Pasta - Dry', 'Global Foods', 'Premade', 'Boxed', 'Italy', 'PastaPerfect', '2023-08-15', 'Durum Wheat', '', '', '', '', '', 1.49, 'PC', '萬安倉', 'Perfect for Italian dishes.'),
    ('Blue Cheese', 'Cheese - Dairy', 'Cheese Crafters', 'Retail', 'Plastic Wrapped', 'France', 'CheeseArt', '2023-09-01', 'Aged 60 days', 'Creamy', 'Strong Flavor', '', '', '', 9.99, 'LB', '送貨', 'Pairs well with wines.'),
    ('Spicy Kimchi', 'Kimchi - Fermented', 'Ferment Valley', 'Retail', 'Jarred', 'Korea', 'KimchiKing', '2023-10-20', 'Napa Cabbage', 'Spicy', 'Vegan', '', '', '', 5.99, 'PC', '萬集倉', 'Authentic Korean taste.')
    ]


for data in sample_data:
    insert_product(connection, data)

Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully


In [24]:
df = pd.read_csv(os.getcwd() + "/sample files/updated金山all.csv" )

In [25]:
df

,Unnamed: 0,productName,productTag,supplier,category,packing,origin,brand,effectiveDate,spec1,spec2,spec3,spec4,spec5,spec6,price,weightUnit,warehouse,notes
0,0,牛坑腩 uk-9012,牛坑腩,金山,Beef,抄,英國,uk-9012,2023-10-30,NaN,NaN,NaN,NaN,NaN,NaN,$ 8 .0,LB,其士,NaN
1,1,excel 牛下肩肉眼卷(1x1) choice,牛下肩肉眼卷,金山,Beef,抄,美國,excel,2023-10-30,choice,1x1,NaN,NaN,NaN,NaN,$ 3 7.5,LB,其士,NaN
2,2,dawn meat 牛胸油,牛胸油,金山,Beef,抄**,愛爾蘭,dawn meat,2023-10-30,NaN,NaN,NaN,NaN,NaN,NaN,$ 2 5.0,LB,其士,NaN
3,3,pantanal 牛頭肉,牛頭肉,金山,Beef,25kg,巴西,pantanal,2023-10-30,NaN,NaN,NaN,NaN,NaN,NaN,$ 8 .5,LB,其士,NaN
4,4,minerva 牛細腩包,牛細腩包,金山,Beef,抄,巴西,minerva,2023-10-30,NaN,NaN,NaN,NaN,NaN,NaN,$ 1 4.8,LB,其士,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9052,9052,鴨舌 hu332,鴨舌,金山,Poultry,10kg,匈牙利,NaN,2024-01-21,u3,NaN,NaN,NaN,NaN,NaN,$ 5 2.0,LB,其士,NaN
9053,9053,鵝半翼 hu105,鵝半翼,金山,Poultry,12kg,匈牙利,NaN,2024-01-21,u10,u1,NaN,NaN,NaN,NaN,$ 8 .5,LB,其士,NaN
9054,9054,鵝腎 hu332,鵝腎,金山,Poultry,15kg,匈牙利,NaN,2024-01-21,u3,NaN,NaN,NaN,NaN,NaN,$ 1 2.0,LB,其士,NaN
9055,9055,鵝腳 hu105,鵝腳,金山,Poultry,12kg,匈牙利,NaN,2024-01-21,u10,u1,NaN,NaN,NaN,NaN,$ 1 8.0,LB,其士,NaN


In [26]:
df = df.drop('Unnamed: 0',axis=1)
dataframe_to_db(df,connection)

Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product inserted successfully
Product in

UnboundLocalError: cannot access local variable 'cursor' where it is not associated with a value

In [4]:
query = "drop table Tradeasy_quotation"
df = query_data_dataframe(connection, query)
display(df)

""


In [13]:
df[['productName', 'price', 'weightUnit', 'warehouse']].to_string()


'               productName  price weightUnit warehouse\n0   Frozen Atlantic Salmon   8.99         KG   嘉里温控貨倉1\n1   Organic Chicken Breast   5.49         LB    沙田冷倉1倉\n2  Angus Beef Ribeye Steak  15.99         KG    其士冷藏倉庫\n3       Jumbo Tiger Shrimp  12.50         LB    亞洲生活冷倉\n4         Mixed Vegetables   2.99         KG       百匯倉\n5          Lamb Loin Chops  18.75         KG       嘉威倉\n6            Pork Sausages   4.99         LB      光輝1倉\n7    Italian Fusilli Pasta   1.49         PC       萬安倉\n8              Blue Cheese   9.99         LB        送貨\n9             Spicy Kimchi   5.99         PC       萬集倉'